# Data Cleaning in Python
The examples and code in this notebook are taken from the [Kaggle Data Cleaning Challenge](https://www.kaggle.com/getting-started/52652)

Detailed explanations for important code snippets are provided by Mervat Abuelkheir as part of the CSEN1095 Data Engineering Course.

The goal of this notebook is to show how to handle missing values and noisy data.

Pay attention to the <span style="color:red"> <b> paragraphs in bold red</b></span>; they ask you to do something and provide input!


## Inspecting the data

First thing we need to do is import a dataset.

<br>We will work in this exercise with the <a href="https://www.kaggle.com/maxhorowitz/nflplaybyplay2009to2016#NFL%20Play%20by%20Play%202009-2017%20(v4).csv"> NFL dataset</a>. The NFL dataset is available in the repository's data folder, and is made available on Kaggle. It contains all the regular season plays from the 2009-2016 NFL seasons. The dataset has 100 columns. Each play contains information on: game situation, players involved, results, and advanced metrics such as expected point and win probability values.

### Important note

<b>The NFL dataset is very big (200+ MB) and is not made available in the `data` folder in this repository. Download the dataset from the link provided, store it in your own data or working directory, and proceed with running the code below, taking care to change your directory according to your own data storage location.</b>

<b>For large datasets, you will be provided with a link through which you can download the data. Another option is to read from the html link, provided the data is well-organized to help with proper parsing.</b>


In [14]:
# modules we'll use
import pandas as pd
import numpy as np

# read in all our data
nfl_data = pd.read_csv("data/NFL Play by Play 2009-2017 (v4).csv")

# set seed for reproducibility
np.random.seed(0) 


FileNotFoundError: [Errno 2] File b'data/NFL Play by Play 2009-2017 (v4).csv' does not exist: b'data/NFL Play by Play 2009-2017 (v4).csv'

In [15]:
# Look at a few rows of the nfl_data file. A handful of missing data!
nfl_data.sample(4)

,6,148,72,35,0,33.6,0.627,50,1
177,5.0,143.0,78.0,0.0,0.0,45.0,0.190,47,0.0
159,4.0,151.0,90.0,38.0,0.0,29.7,0.294,36,0.0
540,3.0,128.0,72.0,25.0,190.0,32.4,0.549,27,1.0
246,0.0,165.0,90.0,33.0,680.0,52.3,0.427,23,0.0


## See how many missing data points we have

Let's see how many we have in each column. Let's start with the NFL dataset. 

In [16]:
# get the number of missing data points per column
missing_values_count_nfl = nfl_data.isnull().sum()

# look at the # of missing points in the first ten columns
print(missing_values_count_nfl)

6        0
148      0
72       0
35       0
0        0
33.6     0
0.627    0
50       0
1        0
dtype: int64


There are a lot of missing values it seems! Let's take a look at the percentage of the missing values in the dataset.

In [17]:
# How many total missing values do we have?
# shape returns the dimentionality of a dataframe (rows and columns), can you guess what product will do?
total_cells_nfl = np.product(nfl_data.shape) 
total_missing_nfl = missing_values_count_nfl.sum()

# percent of data that is missing
percentage_missign_values_nfl = (total_missing_nfl/total_cells_nfl) * 100
print(percentage_missign_values_nfl)

0.0


Almost a quarter of the cells in this dataset are empty! 

In the next step, we're going to take a closer look at some of the columns with missing values and try to figure out what might be going on with them.

## Examining the missing values

This is the point where you should develop some data intuition and look at the data to try and figure out why it is the way it is and how that will affect your analysis. For dealing with missing values, you'll need to use your intution to figure out why the value is missing. One of the most important question you can ask yourself to help figure this out is this:

**Is this value missing becuase it wasn't recorded or becuase it dosen't exist?**

If a value is missing becuase it doens't exist (like the height of the oldest child of someone who doesn't have any children) then it doesn't make sense to try and guess what it might be. These values you probably do want to keep as `NaN`. On the other hand, if a value is missing becuase it wasn't recorded, then you can try to guess what it might have been based on the other values in that column and row. (This is called "imputation" and we'll learn how to do it next! :)

Let's work through an example. Looking at the number of missing values in the nfl_data dataframe, notice that the column TimesSec has a lot of missing values in it: 

In [18]:
# look at the # of missing points in the first ten columns
missing_values_count_nfl[0:10]

6        0
148      0
72       0
35       0
0        0
33.6     0
0.627    0
50       0
1        0
dtype: int64

The `TimeSecs` column has information on the number of seconds left in the game when the play was made. This means that these values are probably missing because they were not recorded, rather than because they don't exist. So, it would make sense for us to try and guess what they should be rather than just leaving them as NA's.

On the other hand, there are other fields, like `PenalizedTeam` that also have lot of missing fields. In this case, though, the field is missing because if there was no penalty then it doesn't make sense to say which team was penalized. For this column, it would make more sense to either leave it empty or to add a third value like "neither" and use that to replace the NA's.

**Tip: This is a great place to read over the dataset documentation if you haven't already! If you're working with a dataset that you've gotten from another person, you can also try reaching out to them to get more information.**

You should look at each column individually to figure out the best strategy for filling those missing values. For the rest of this notebook, we'll cover some "quick and dirty" techniques that can help you with missing values but will probably also end up removing some useful information or adding some noise to your data.

## Dropping the missing values

If you don't have a reason to figure out why your values are missing, one option you have is to just remove any rows or columns that contain missing values. (Note: DO NOT do this approch for important projects! It's usually worth it to take the time to go through your data and really look at all the columns with missing values one-by-one to really get to know your dataset.)  

If you're sure you want to drop rows with missing values, pandas does have a handy function, `dropna()` to help you do this. Let's try it out on our NFL dataset!

In [19]:
# remove all the rows that contain a missing value
nfl_data.dropna()

,6,148,72,35,0,33.6,0.627,50,1
0,1.0,85.0,66.0,29.0,0.0,26.6,0.351,31,0.0
5,3.0,78.0,50.0,32.0,88.0,31.0,0.248,26,1.0
7,2.0,197.0,70.0,45.0,543.0,30.5,0.158,53,1.0
12,1.0,189.0,60.0,23.0,846.0,30.1,0.398,59,1.0
13,5.0,166.0,72.0,19.0,175.0,25.8,0.587,51,1.0
18,1.0,115.0,70.0,30.0,96.0,34.6,0.529,32,1.0
23,11.0,143.0,94.0,33.0,146.0,36.6,0.254,51,1.0
24,10.0,125.0,70.0,26.0,115.0,31.1,0.205,41,1.0
30,3.0,158.0,76.0,36.0,245.0,31.6,0.851,28,1.0
38,4.0,111.0,72.0,47.0,207.0,37.1,1.390,56,1.0


In [20]:
from sklearn.preprocessing import Imputer
#from sklearn.impute import SimpleImputer # pretty much the same thing, you can try yourself
mean_imputer = Imputer(missing_values=np.nan, strategy='mean')
#mean_simpleimputer = SimpleImputer(missing_values=np.nan, strategy='mean') # Same use for sklearn SimpleImputer

# Again, let's get a small subset of the NFL dataset
subset_nfl_data = nfl_data.loc[:, 'EPA':'Season'].head()

# Check the presence of null values in the subset chosen
subset_nfl_data.isnull().sum()


/Users/hanienelgalfy/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


KeyError: 'EPA'

Oops! That's removed all our data! <span style="color:red"><b>Why do you think this happened?</b></span>

We might have better luck removing all the columns that have at least one missing value instead.

In [21]:
# Remove all columns with at least one missing value
columns_with_na_dropped = nfl_data.dropna(axis=1) # axis 1 indicates operation along columns
columns_with_na_dropped.head()

,6,148,72,35,0,33.6,0.627,50,1
0,1.0,85.0,66.0,29.0,0.0,26.6,0.351,31,0.0
5,3.0,78.0,50.0,32.0,88.0,31.0,0.248,26,1.0
7,2.0,197.0,70.0,45.0,543.0,30.5,0.158,53,1.0
12,1.0,189.0,60.0,23.0,846.0,30.1,0.398,59,1.0
13,5.0,166.0,72.0,19.0,175.0,25.8,0.587,51,1.0


Let's check how much data we lost.

In [22]:
print("Columns in original dataset: %d \n" % nfl_data.shape[1])
print("Columns with na's dropped: %d" % columns_with_na_dropped.shape[1])

Columns in original dataset: 9 

Columns with na's dropped: 9


We've lost quite a bit of data, but at this point we have successfully removed all the `NaN`'s from our data. 

# Filling in missing values automatically

Another option is to try and fill in the missing values. Let's get a small sub-section of the NFL data so that it will print well and work with that to avoid having to work with the entire dataset and consume CPU time. You can work on the entire dataset at home!

In [23]:
# get a small subset of the NFL dataset
subset_nfl_data = nfl_data.loc[:, 'EPA':'Season'].head()
subset_nfl_data

KeyError: 'EPA'

We can use the Panda's fillna() function to fill in missing values in a dataframe for us. One option we have is to specify what we want the `NaN` values to be replaced with. Here, I'm saying that I would like to replace all the `NaN` values with 0.

In [24]:
# replace all NA's with 0
subset_nfl_data.fillna(0) 

NameError: name 'subset_nfl_data' is not defined

We could also be a bit more savvy and replace missing values with whatever value comes directly after it in the same column. (This makes a lot of sense for datasets where the observations have some sort of logical order to them.)

In [25]:
# replace all NA's the value that comes directly after it in the same column, 
# then replace all the reamining na's with 0
subset_nfl_data.fillna(method = 'bfill', axis=0).fillna(0) # here you may want to explore other methods like 'mean' if it makes sense

NameError: name 'subset_nfl_data' is not defined

## Trying multivariate imputation with sklearn

sklearn has an `Imputer` and a `SimpleImputer` modules which perform imputation on one column at a time using the mean, median, or mode. 

sklearn also has a multivariate imputation module, `IterativeImputer`, that can help us impute values across multiple columns, but it is experimental and only available through developer release.

Let's see how an sklearn's imputation works for the NFL dataset.

In [26]:
# Now let's impute missing values in yacWPA column with the mean
mean_imputer.fit(subset_nfl_data[["yacWPA"]])
subset_nfl_data[["yacWPA"]]=mean_imputer.transform(subset_nfl_data[["yacWPA"]]).ravel()
#imputed_subset_nfl_data = pd.DataFrame(mean_simpleimputer.fit_transform(subset_nfl_data), columns=subset_nfl_data.columns) # fitting and transforming can be done in one step

# Let's check the presence null values again to see if imputation worked for the column
subset_nfl_data.isnull().sum()
# add you own counting of nulls for the dataframe imputed by SimpleImputer


NameError: name 'subset_nfl_data' is not defined

In [27]:
# Print the dataframe 
print(subset_nfl_data)
#print(imputed_subset_nfl_data) # if you want to try SimpleImputer code
# You may want to try the same code snippets above with the entire dataset and not just the subset

NameError: name 'subset_nfl_data' is not defined

Now let's try the more advanced `IterativeImputer` in sklearn. It provides means to do multivariate imputation using a variety of ML models.

In [28]:
# This code snippet may not work, so do not worry about it, and just examine the similarities with the SimpleImputer
# To use this experimental feature, we need to explicitly ask for it:
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge # Imputation via linear regression
from sklearn.neighbors import KNeighborsRegressor # Imputation via KNN
# Define imputer
reg_imputer = IterativeImputer(BayesianRidge, max_iter=5, random_state=0)
subset_nfl_data = nfl_data.loc[:, 'EPA':'Season'].head()
imputed_subset_nfl_data = pd.DataFrame(mean_simpleimputer.fit_transform(subset_nfl_data), columns=subset_nfl_data.columns)
print(imputed_subset_nfl_data)

KeyError: 'EPA'

## <span style="color:red"> Exercise 1: Missing value handling on the San Francisco Building Permits Dataset </span>

<span style="color:red"><b> The <a href="https://www.kaggle.com/aparnashastry/building-permit-applications-data#Building_Permits.csv"> Building permits dataset</a> pertains to all types of structural permits from Jan 1, 2013-Feb 25th 2018 in San Francisco. Data includes details on application/permit numbers, job addresses, supervisorial districts, and the current status of the applications. A data dictionary for the dataset is available in the data folder in this repository.</b></span>

<span style="color:red"> <b>The dataset is rather big and does not exist in the "data" folder in this repository. Download the dataset from the link provided and store it in your own data or working folder. Import the dataset in the next cell, and work your way through the steps in the subsequent cells.</b></span>

<span style="color:red"> <b>Find out what percent of the building permits dataset is missing. </b></span>
<span style="color:red"> <b><br>Write down your code in the following cell. </b></span>

<span style="color:red"><b>Write a code to check the values of the `Street Number Suffix` and `Zipcode` from the building permits datasets. Both of these contain missing values. Which, if either, of these are missing because they don't exist? Which, if either, are missing because they weren't recorded?</b></span>

<span style="color:red"><b>See if you can drop the missing values from the building permits dataset and still keep some data, or drop the columns with missing values and see if you have columns still.</b></span>

<span style="color:red"><b>Try replacing all the `NaN`'s in the building permits data with the one that comes directly after it and then replacing any remaining `NaN`'s with 0.</b></span>

<span style="color:red"><b>Create a new dataset at the ending with all missing values handled and store as a CSV file.</b></span>

## <span style="color:red"> Exercise 2: Missing value handling on the  Pima Indians Diabetes Dataset</span>

<span style="color:red"><b> The <a href= "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"> Pima Indians Diabetes Dataset</a> involves predicting the onset of diabetes within 5 years in Pima Indians given medical details.</b></span>

<span style="color:red">There are 768 observations with 8 input variables and 1 output variable. The variable names are as follows:</span>

<br><span style="color:red">0. Number of times pregnant.</span>
<br><span style="color:red">1. Plasma glucose concentration a 2 hours in an oral glucose tolerance test.</span>
<br><span style="color:red">2. Diastolic blood pressure (mm Hg).</span>
<br><span style="color:red">3. Triceps skinfold thickness (mm).</span>
<br><span style="color:red">4. 2-Hour serum insulin (mu U/ml).</span>
<br><span style="color:red">5. Body mass index (weight in kg/(height in m)^2).</span>
<br><span style="color:red">6. Diabetes pedigree function.</span>
<br><span style="color:red">7. Age (years).
<br><span style="color:red">8. Class variable (0 or 1).</span>

<span style="color:red"> <b>This is a medical dataset, so an important thing we will do with this dataset is that we will see if the statistical summaries can reveal something about the missing values.</b></span>

<span style="color:red"> <b>The dataset is available in the `data` folder in this repository. Import the dataset in the next cell, and work your way through the steps in the subsequent cells.</b></span>

In [29]:
import pandas as pd
import numpy as np

nfl_data = pd.read_csv("data/pima-indians-diabetes.data.csv")
np.random.seed(0) 

nfl_data.sample(5)

,6,148,72,35,0,33.6,0.627,50,1
661,8,167,106,46,231,37.6,0.165,43,1
122,5,132,80,0,0,26.8,0.186,69,0
113,7,160,54,32,175,30.5,0.588,39,1
14,7,100,0,0,0,30.0,0.484,32,1
529,2,122,60,18,106,29.8,0.717,22,0


<span style="color:red"> <b>Use the describe method used for pandas data frames to obtain a statistucal summary of the dataset.</b></span>

In [30]:
nfl_data.dropna(inplace = True)  
perc =[.20, .40, .60, .80] 
include =['object', 'float', 'int'] 
desc = nfl_data.describe(percentiles = perc , include = include) 
desc 

,6,148,72,35,0,33.6,0.627,50,1
count,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000
mean,3.842243,120.859192,69.101695,20.517601,79.903520,31.990482,0.471674,33.219035,0.348110
std,3.370877,31.978468,19.368155,15.954059,115.283105,7.889091,0.331497,11.752296,0.476682
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
20%,1.000000,95.000000,60.000000,0.000000,0.000000,25.900000,0.219200,23.000000,0.000000
40%,2.000000,109.000000,68.000000,18.000000,0.000000,30.100000,0.302400,27.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,32.000000,32.000000,0.371000,29.000000,0.000000
60%,4.000000,125.000000,74.000000,27.000000,72.600000,33.700000,0.453600,33.000000,0.000000
80%,7.000000,146.800000,82.000000,35.000000,150.000000,37.800000,0.687000,42.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


<span style="color:red"><b>There are columns that have a minimum value of zero (0). On some columns, a value of zero does not make sense and indicates an invalid or missing value. For example, the `BMI` for a person cannot be 0. See if there are other columns/attributes whose value of 0 does not make sense. An understanding of the medical columns may be needed.</b></span>

In [31]:
import numpy
nfl_data[1:148] = nfl_data[1:480].replace(0,numpy.NaN)
nfl_data.dropna(inplace = True)
nfl_data.describe()

,6,148,72,35,0,33.6,0.627,50,1
count,642.000000,642.000000,642.000000,642.000000,642.000000,642.000000,642.000000,642.000000,642.000000
mean,3.817757,122.454829,69.266355,21.155763,86.528037,32.252336,0.474369,33.403427,0.369159
std,3.386142,32.303002,19.167032,15.873742,120.482053,7.549665,0.326580,11.847346,0.482953
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,100.000000,64.000000,0.000000,0.000000,27.500000,0.243250,24.000000,0.000000
50%,3.000000,119.000000,72.000000,23.000000,48.500000,32.000000,0.380000,29.000000,0.000000
75%,6.000000,142.750000,80.000000,33.000000,135.000000,36.600000,0.635500,41.000000,1.000000
max,17.000000,199.000000,114.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


<span style="color:red"><b>Now you can proceed to check the missing values and remove/impute as in exercise 1. Create a new dataset at the ending with all missing values handled and store as a CSV file.</b></span>

In [32]:
from pandas import DataFrame
df = DataFrame(nfl_data, columns= ["pregnant","plasma-glucose","blood-pressure","triceps-thickness","insulin","BMI","Diabetes-pedigree","age","class"])

export_csv = df.to_csv (r'/Users/hanienelgalfy/Documents/GUC/Data\ Engineering/CSEN1095-Data-Engineering/export.csv', index = None, header=True)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/hanienelgalfy/Documents/GUC/Data\\ Engineering/CSEN1095-Data-Engineering/export.csv'